# Enitt Proj
## 환경센서를 통한 발전량 예측

    
    1. Project intro
    2. Enterprise data set
    3. Exploratory data analysis
    4. data crawling
    5. data preprocessing
    6. modeling and training
    7. validation, evaluation
    

### 0) load module

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import datetime
import seaborn as sns 
import scipy.stats
import plotly.express as px


### 1. Project intro

##### Enitt발전소에 맞춘 맞춤형 태양광 발전량을 예측한다.
##### 현재 https://bd.kma.go.kr/kma2020/fs/energySelect1.do?pageNum=5&menuCd=F050701000 처럼
##### 각 지역에 맞는 일사량에 따른 발전량 예측사이트가 있지만, 
##### 태양광 발전량은 주변 지역 환경과 설비용량, 각 인버터 스펙에 따른 민감도가 존재한다.
     즉 대략적인 발전량 예측은 가능하지만 설비용량과 출력스펙을 고려한 발전량은 예측이 불가능하다.

##### 이 프로젝트에서는 Enitt 발전소별 일사량 보정값과 가장 인접해 있는 지역의 환경데이터를 통하여 
##### 각 발전소별 모델링을 구축하여 발전소 맞춤형 태양광 발전량 예측 모델을 설계한다.

### 2. Enterprise data set


##### 각 지역별 발전량 정보 및 환경센서 정보


In [2]:
ent_data_set1 = pd.read_csv("./ENITT_SOLAR_DATA/ENITT_SOLAR_DATA/365. 반포써밋2블럭 발전량.csv", encoding="ms949")
ent_data_set2 = pd.read_csv("./ENITT_SOLAR_DATA/ENITT_SOLAR_DATA/365. 반포써밋2블럭 환경정보.csv", encoding="ms949")

In [3]:
ent_data_set1.head(5)

,pow_index,pow_id,pow_date,pow_time,pow_cap,pow_dcv,pow_dca,pow_dcp,pow_acvr,pow_acvs,pow_acvt,pow_acar,pow_acas,pow_acat,pow_acp,pow_pf,pow_totpower,pow_freq
0,1,2,2018-10-18,14:35:09,15.0,439,10.0,4300.0,375,372,375,6.9,7.4,7.2,4100.0,0.9,9000,60.0
1,2,2,2018-10-18,14:36:21,15.0,435,9.8,4200.0,374,371,376,6.7,7.3,7.1,4000.0,0.9,9000,59.9
2,3,2,2018-10-18,14:37:18,15.0,444,9.6,4200.0,375,372,376,6.6,7.3,7.1,4000.0,0.9,9000,59.9
3,4,2,2018-10-18,14:38:10,15.0,453,9.3,4200.0,376,373,377,6.5,7.1,6.8,4000.0,0.9,9000,60.0
4,5,2,2018-10-18,14:39:03,15.0,454,9.2,4100.0,376,372,376,6.5,7.2,7.1,3900.0,0.9,9000,59.9


In [4]:
ent_data_set2.head(5)

,env_index,env_date,env_time,env_slopesolar,env_levelsolar,env_modtemp,env_airtemp
0,1,2018-10-22,12:44:29,322,316,24.8,17.6
1,2,2018-10-22,12:45:21,320,314,25.0,17.7
2,3,2018-10-22,12:46:12,320,312,25.1,17.9
3,4,2018-10-22,12:47:04,326,319,24.9,18.1
4,5,2018-10-22,12:47:56,335,329,24.9,18.0


### 3. Exploratory data analysis

#### 1) Enterprise data
      1-1 발전량(energy generation)
       : 전류 x 전압 = 에너지 
       
       * 발전량 정보 데이터의 pow_dcp, pow_totpower 컬럼이 에너지를 나타내는 부분이나, 센서의 민감도나 측정단위 문제로 정확한 측정이 안된다.
         pow_dcp의 측정 최소 단위는 100W이고 pow_totpower의 측정 최소단위는 1000W이고 
         pow_dcp는 149W까지는 측정이 불가하고 150W부터 반내림한 100W로 측정을 하게 된다 (pow_totpower 이와 동일)

         하여 정확한 발전량 측정을 하기 위하여 직류전류(pow_dcv) X 직류전압(pow_dca) 를 발전량 칼럼인 Watt로 생성하였다.
         


In [11]:
print("전력", ent_data_set1["pow_dcp"].unique()[:5])
print("누적발전량", ent_data_set1["pow_totpower"].unique()[:5])

전력 [4300. 4200. 4100. 9900. 3800.]
누적발전량 [   9000 2936000   10000   11000   12000]
